# Sentinel-2 cropland mapping

Following the paper by [Belgiu & Csillik (2018)] (see also [Hao et al. 2018](https://peerj.com/articles/5431/?utm_source=TrendMD&utm_campaign=PeerJ_TrendMD_0&utm_medium=TrendMD))(https://www.sciencedirect.com/science/article/pii/S0034425717304686) we can train a CNN for the segmentation of the croplands. As an input we can use [Sentinel-2 MSI](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) multispectral data, and as an output crop types data classified by experts from the European Land Use and Coverage Area Frame Survey ([LUCAS](https://ec.europa.eu/eurostat/statistics-explained/index.php/LUCAS_-_Land_use_and_land_cover_survey)) and  CropScape – Cropland Data Layer ([CDL](https://nassgeodata.gmu.edu/CropScape/)), respectively.

Datasets in Google Earth Engine:

- [Sentinel-2 MSI: MultiSpectral Instrument, Level-1C](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2)
- [USDA NASS Cropland Data Layers](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL)
- [Canada AAFC Annual Crop Inventory](https://developers.google.com/earth-engine/datasets/catalog/AAFC_ACI) 
***

## Configure the Environment

We begin by importing a number of useful libraries

In [1]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image
from functools import reduce
import h5py

Initialize the Earth Engine client.

In [2]:
ee.Initialize()

## Download datasets
We download and append datasets from different Areas of Interest (AOIs)

In [5]:
from ee_datasets import ee_datasets

In [113]:
# Central position of (AOIs)
points = [[-120.7224, 37.3872], [-112.6799, 42.9816], [-89.7649, 35.8764], 
          [-96.0181, 41.2412], [-115.473, 46.861], [-103.9803, 47.9713], [-96.9217, 32.8958]]

# Start and stop of time series
startDate = ee.Date('2016-01-01')
stopDate  = ee.Date('2016-12-31')
# Scale in meters
scale = 10
# Buffer
buffer = 15000

In [118]:
dpix = 250
for n, point in enumerate(points):
    sentinel = ee_datasets(point = point, buffer = buffer, startDate = startDate, stopDate = stopDate, scale = scale, collection = 'Sentinel2')
    cropland = ee_datasets(point = point, buffer = buffer, startDate = startDate, stopDate = stopDate, scale = scale, collection = 'CroplandDataLayers')
    dataset_x = sentinel.read_datasets()
    dataset_y = cropland.read_datasets()
    
    if n == 0:
        dim_x = dataset_x.shape
        dim_x = list(dim_x)
        dim_x = [1] + dim_x
        dim_x[1] = dim_x[1]-dpix; dim_x[2] = dim_x[2]-dpix
        
        dim_y = dataset_y.shape
        dim_y = list(dim_y)
        dim_y = [1] + dim_y
        dim_y[1] = dim_y[1]-dpix; dim_y[2] = dim_y[2]-dpix
        
        data_x = np.zeros(dim_x, dtype=np.float32)
        data_y = np.zeros(dim_y, dtype=np.float32)
        
        data_x[0,:] = dataset_x[:dim_x[1],:dim_x[2],:dim_x[3]]
        data_y[0,:] = dataset_y[:dim_y[1],:dim_y[2],:dim_y[3]]
    else:
        data_x2 = np.zeros(dim_x, dtype=np.float32)
        data_y2 = np.zeros(dim_y, dtype=np.float32)
        
        data_x2[0,:] = dataset_x[:dim_x[1],:dim_x[2],:dim_x[3]]
        data_y2[0,:] = dataset_y[:dim_y[1],:dim_y[2],:dim_y[3]]
        
        data_x = np.append(data_x, data_x2, axis=0)
        data_y = np.append(data_y, data_y2, axis=0)

url:  https://earthengine.googleapis.com/api/download?docid=e1799180a8edf3be1ff76a00883f4ee9&token=639cb50f6cbdc23c82897b7c67f07364
Download complete!
url:  https://earthengine.googleapis.com/api/download?docid=4ce69f77412f23e0518f945bc541abbf&token=133389ed21c1776ba2475278034f8340
Download complete!
url:  https://earthengine.googleapis.com/api/download?docid=55d61466d74554ca28cfa03aa2ea8436&token=d79e5ecde5ab1bb36c28eec9517c2b3c
Download complete!
url:  https://earthengine.googleapis.com/api/download?docid=60af59604bfadbeb08c6b174f93b92a8&token=694e3d00b6cf80157ea7d43189e15188
Download complete!
url:  https://earthengine.googleapis.com/api/download?docid=2a9d9fada6b5b6db16b1615440c705fb&token=69e385cd70f9ddee7126ac7558ea3387
Download complete!
url:  https://earthengine.googleapis.com/api/download?docid=2c4758b7785c5377a50798527a0518aa&token=38be43ddb77ab5b2aebeb0cc99567529
Download complete!
url:  https://earthengine.googleapis.com/api/download?docid=2ec7186e80827c8853f2c42c3dfb23be&t

## Preprocess class labels

Each class in encoded as a value in the range between 0 to 254. For training a Neural Network in Keras we have to convert the 1-dimensional class arrays to N classes-dimensional matrices. To simplify the problem here we regroup all the classes into 4 categories, namely, land, water, urban, and cropland areas.

In [172]:
from preprocess_data import categorical_data

In [392]:
new_data_y = categorical_data(data_y)

## Normalize input data
Normalize the input channels to the range [0,1]

In [174]:
from preprocess_data import normalize_data

In [393]:
new_data_x = normalize_data(data_x)

## Resize the images

In [ ]:
from preprocess_data import resize_patches

In [394]:
new_data_x, new_data_y = resize_patches(new_data_x, new_data_y, patch_size=128)

## Randomize the datasets

In [350]:
from preprocess_data import randomize_datasets

In [401]:
x_randm, y_randm = randomize_datasets(new_data_x, new_data_y)

## Training and validation sets

In [376]:
from preprocess_data import train_validation_split

In [402]:
x_train, x_validation, y_train, y_validation = train_validation_split(x_randm, y_randm)

## Save datasets

In [313]:
from preprocess_data import write_data

In [413]:
# Save samples
write_data("../SegNet/Samples/x_train.h5", 'x_train', x_train)
write_data("../SegNet/Samples/y_train.h5", 'y_train', y_train)
write_data("../SegNet/Samples/x_validation.h5", 'x_validation', x_validation)
write_data("../SegNet/Samples/y_validation.h5", 'y_validation', y_validation)

## Train the Network 

To train SegNet from the command line type:

```python
python train_SegNet.py -i Samples/ -o Network/SegNet -e 20 -a start
```

The parameters are:

```
-i = Samples/model 
    Path of the input files.
-o = network/model 
    Path of the output file that will contain the network weights.
-e = 20
    Number of epochs to use during training.
-a = {start,continue}
     `start`: start a new calculation
     `continue`: continue a previous calculation
```